In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [20]:
def load_image(image_path):
    img = io.imread(image_path)
    return img

In [21]:
normal_paths = [os.path.join("../abnormal", filename) for filename in os.listdir("../abnormal")]
abnormal_paths = [os.path.join("../Normal Gait", filename) for filename in os.listdir("../Normal Gait")]

In [22]:
normal_images = [load_image(image_path) for image_path in normal_paths]
abnormal_images = [load_image(image_path) for image_path in abnormal_paths]

In [23]:
def extract_hog_features(images):
    features = []
    for img in images:
        hog_features = hog(img, block_norm='L2-Hys', pixels_per_cell=(16, 16), cells_per_block=(1, 1))
        features.append(hog_features)
    return features

In [24]:
X_normal = extract_hog_features(normal_images)
X_abnormal = extract_hog_features(abnormal_images)

In [25]:
X = X_normal + X_abnormal
y = ['normal'] * len(X_normal) + ['abnormal'] * len(X_abnormal)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
num_rbf_centers = 25
kmeans = KMeans(n_clusters=num_rbf_centers)
kmeans.fit(X_train)
rbf_centers = kmeans.cluster_centers_

c:\Gait Analysis\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [28]:
def gaussian_rbf(x, center, width):
    return np.exp(-width * np.linalg.norm(x - center)**2)

In [29]:
def calculate_rbf_activations(data, centers, width):
    activations = np.zeros((len(data), len(centers)))
    for i in range(len(centers)):
        for j in range(len(data)):
            activations[j, i] = gaussian_rbf(data[j], centers[i], width)
    return activations

In [30]:
rbf_width = 0.01
rbf_activations_train = calculate_rbf_activations(X_train, rbf_centers, rbf_width)
rbf_activations_test = calculate_rbf_activations(X_test, rbf_centers, rbf_width)

In [31]:
clf = SVC(C=0.1)
clf.fit(rbf_activations_train, y_train)

SVC(C=0.1)

In [32]:
y_pred_train = clf.predict(rbf_activations_train)
y_pred_test = clf.predict(rbf_activations_test)

In [36]:
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"Training Accuracy: {accuracy_train:.2f}")
print(f"Testing Accuracy: {accuracy_test:.2f}")

Training Accuracy: 0.62
Testing Accuracy: 0.61


In [35]:
# calculate cross entropy loss
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-10
    loss = 0
    for i in range(len(y_true)):
        if y_true[i] == 'normal':
            loss += -np.log(1 - y_pred[i] + epsilon)
        else:
            loss += -np.log(y_pred[i] + epsilon)
    return loss / len(y_true)

y_pred_train_proba = clf.decision_function(rbf_activations_train)
y_pred_test_proba = clf.decision_function(rbf_activations_test)

y_pred_train_proba = 1 / (1 + np.exp(-y_pred_train_proba))
y_pred_test_proba = 1 / (1 + np.exp(-y_pred_test_proba))

loss_train = cross_entropy_loss(y_train, y_pred_train_proba)
loss_test = cross_entropy_loss(y_test, y_pred_test_proba)

print(f"Training Loss: {loss_train:.2f}")
print(f"Testing Loss: {loss_test:.2f}")

Training Loss: 0.91
Testing Loss: 0.90
